<a href="https://colab.research.google.com/github/crispitagorico/Neural-SPDEs/blob/main/Parabolic1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone "https://github.com/crispitagorico/Neural-SPDEs.git"

In [ ]:
%cd Neural-SPDEs/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
import torch.optim as optim

from utilities import MatReader, LpLoss, count_params
from neuralSPDE_1D import NeuralFixedPoint

torch.manual_seed(0)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
reader = MatReader('../drive/MyDrive/parabolic_multiplicative.mat')
xi = reader.read_field('forcing')
data = reader.read_field('sol')

In [ ]:
ntrain = 800
ntest = 200

batch_size = 20
epochs = 500
learning_rate = 0.0025
scheduler_step = 100
scheduler_gamma = 0.5
step = 1

sub_x = 1
sub_t = 50

u0_train = data[:ntrain,::sub_x,0][:,None,...]
u_train = data[:ntrain,::sub_x,1::sub_t]
xi_train = xi[:ntrain, ::sub_x, 1::sub_t][:,None,...]

u0_test = data[-ntest:,::sub_x,0][:,None,...]
u_test = data[-ntest:,::sub_x,1::sub_t]
xi_test = xi[-ntest:, ::sub_x, 1::sub_t][:,None,...]

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(u0_train, xi_train, u_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(u0_test, xi_test, u_test), batch_size=batch_size, shuffle=False)

In [ ]:
# model = NeuralFixedPoint(modes1=16, modes2=16, modes3=8, in_channels=1, hidden_channels=64, forcing_channels=1, out_channels=1, T=T, n_iter=4).cuda()
model = NeuralFixedPoint(modes1=8, modes2=8, in_channels=1, hidden_channels=32, forcing_channels=1, out_channels=1, T=u_train.shape[2], n_iter=8).cuda()

print(count_params(model))

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [ ]:
myloss = LpLoss(size_average=False)

for ep in range(epochs):

    model.train()
    
    train_loss = 0.
    for u0_, xi_, u_ in train_loader:

        loss = 0.

        u0_ = u0_.to(device)
        xi_ = xi_.to(device)
        u_ = u_.to(device)

        u_pred = model(u0_, xi_)

        loss = myloss(u_pred.reshape(batch_size, -1), u_.reshape(batch_size, -1))

        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    
    test_loss = 0.
    with torch.no_grad():
        for u0_, xi_, u_ in test_loader:
            
            loss = 0.
            
            u0_ = u0_.to(device)
            xi_ = xi_.to(device)
            u_ = u_.to(device)

            u_pred = model(u0_, xi_)

            loss = myloss(u_pred.reshape(batch_size, -1), u_.reshape(batch_size, -1))

            test_loss += loss.item()

    scheduler.step()
    print('Epoch {:04d} | Total Train Loss {:.6f} | Total Test Loss {:.6f}'.format(ep, train_loss / ntrain, test_loss / ntest))
    

**Visualize results**

In [ ]:
k = 5
T_= 10

# u0 = u0_train[k][None,...].to(device)
# u = u_train[k][None,...].to(device) 
# xi = xi_train[k][None,...].to(device)
u0 = u0_test[k][None,...].to(device)
u = u_test[k][None,...].to(device) 
xi = xi_test[k][None,...].to(device)
u_pred = model(u0,xi)

fig, ax = plt.subplots(1,T_+1,figsize=(25,3))
plt.suptitle('Parabolic Multiplicative Forcing', fontsize=16)

ax[0].plot(u0[0,0,...].detach().cpu().numpy())
ax[0].set_title('initial condition')

for i in range(T_):
  ax[i+1].plot(u[0,...,i].detach().cpu().numpy())
  ax[i+1].plot(u_pred[0,0,...,i].detach().cpu().numpy())
  # ax[0][i+1].set_title(f'time step {i+1}')
plt.tight_layout()
plt.show()